# Part 2: Simulating with DeerLab models

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import deerlab as dl

### Learning Objectives of Part 2

- To obtain a basic understanding of the structure of models in DeerLab 
- To be able to simulate simple dipolar signals for experiments such as 4-pulse DEER
- To understand how to simulate individual componets of a dipolar signals such as distance distributions, or background functions.

---------------------

## 1. DeerLab models, the basics

All DeerLab models consist of a set of parameters. Each parameter will always have the following properties that can be modified at any point (we will see how later):

- ``Start``
    The start values at which the parameters search start during fitting and optimization. 

- ``Lower/Upper``
    The upper/lower bounds of the parameter that constrain the parameter values during fitting and optimization.

- ``Frozen``
    Whether a parameter is frozen to a specific value. Freezing refers to setting a parameter to a static value and to be omitted during the fitting routines. 

- ``Constants`` Any quantities required by the model to be calculated but that are not considered parameters and thus kept constant during the fitting routines.

- ``Signature`` The order in which to pass the parameters in order to evaluate the model at specific values. 

You can quickly and easily inspect all the model properties by displaying the model's summary using the ``print()`` function.  

In [ ]:
# Print the summary of the Gaussian distance distirbution model
print(dl.dd_gauss)

In [ ]:
# Define vector of interspin distances
r = np.arange(1,10,0.05) # nm, 0.05nm resolution

rmean = 4 # Mean distance, nm
rstd = 0.3 # Standard deviation of distances, nm  

# Evaluate the model using positional arguments (order of arguments is important)
P = dl.dd_gauss(r,rmean,rstd)

# EValuate the model using keyword arguments (order of arguments is not important)
P = dl.dd_gauss(mean=rmean, std=rstd, r=r)

plt.plot(r,P,'b')
plt.xlabel('r [nm]')
plt.ylabel('P(r) [nm$^{-1}$]')
plt.show()

The same applies to background models

In [ ]:
# Print the summary of the Gaussian distance distirbution model
print(dl.bg_hom3d)

In [ ]:
# Define vector of interspin distances
t = np.arange(-1,5,0.008) # μs, 8ns resolution

conc = 150 # Spin concentration, μM
mod = 0.3 # Modulation depth 

# Evaluate the model using positional arguments (order of arguments is important)
B = dl.bg_hom3d(t,conc,mod)

# EValuate the model using keyword arguments (order of arguments is not important)
B = dl.bg_hom3d(lam=mod, conc=conc, t=t)

plt.plot(t,B,'b')
plt.xlabel('t [μs]')
plt.ylabel('B(t)')
plt.show()

**Remember:** All dipolar EPR-related models in DeerLab define **time units in microseconds** and **distance units in nanometers**!


You can get more detailed information and tutorials on how DeerLab's models work and how to contruct you own models using the [following documentation](https://jeschkelab.github.io/DeerLab/modelling_guide.html). 

-----------

## Hands-on #1: Simulating a distance distribution

Simulate a distance distribution $P(r)$ modeled as bimodal Gaussian distribution with one Gaussian centered about 5.4nm with a standard deviation of 0.42nm and the other with Gaussian centered about 4.4nm with a standard deviation of 0.25nm. The two Gaussians have an amplitude ratio of 1:2.

First, check the details on the model by printing it.

In [ ]:
# <your script goes here>

Next, simulate the distribution and make a plot $r$ vs $P(r)$ using the Matplotlib package.

In [ ]:
# <your script goes here>

-------------

# 2. Dipolar signal models

To construct a model of a dipolar signal, DeerLab provides the ``dipolarmodel`` function, which will contruct different dipolar models based on your needs. The minimum information that you need to provide to ``dipolarmodel`` is the vector of dipolar evolution times $t$ and the vector of interspin distances $r$.  By doing so, ``dipolarmodel`` will construct a single-pathway dipolar model (the same model used in all other software such as DeerAnalysis, LongDistances, etc.) with a non-parametric distance distribution.

In [ ]:

rmin = 1  # nm
rmax = 7 # nm
dr = 0.05 # nm
r = np.arange(rmin,rmax,dr)

tau1 = 0.5 # μs
tau2 = 6.0 # μs
dt = 0.008 # μs
t = np.arange(0,tau1+tau2,dt)

# Construct a single pathway model
Vmodel = dl.dipolarmodel(t,r)
# This is equivalent to... 
Vmodel = dl.dipolarmodel(t,r, Pmodel=None, Bmodel=dl.bg_hom3d)

print(Vmodel)


The function ``dipolarsignal`` allows you to easily switch between different models of the distance distribution and the background. These can be specified via the ``Pmodel`` and ``Bmodel`` arguments. 

In [ ]:
# Dipolar signal model with a Gaussian distance distribution and a homogeneous 3D-background model
Vmodel = dl.dipolarmodel(t,r,Pmodel=dl.dd_gauss)
print(Vmodel)

In [ ]:
# Dipolar signal model with a non-paremtric distance distribution and an exponential phenomenological background 
Vmodel = dl.dipolarmodel(t,r,Bmodel=dl.bg_exp)
print(Vmodel)

In [ ]:
# Dipolar signal model with a random coil distance distribution and a phenomenological stretched exponential
Vmodel = dl.dipolarmodel(t,r,Pmodel=dl.dd_randcoil,Bmodel=dl.bg_strexp)
print(Vmodel)

Now, let's simulate a 4-pulse DEER signal measured on some biradical system with an interspin distance distributed according to a worm-like chain model (contour length of 3.6nm and a persistence length of 9nm) with a spin concentration of 50μM. Since it is a 4-pulse DEER experiment, the main pathway contribution's refocusing time (i.e. zero-time) must be at $t=\tau_1$ and we will assume a modulation depth of 35%.  

In [ ]:
# Model parameters
persistence = 5 # nm
contour = 3.6 # nm
moddepth = 0.35 
reftime = tau1 # μs
spinconc = 50  # μM

# Construct the dipolar signal model 
Vmodel = dl.dipolarmodel(t,r,Pmodel=dl.dd_wormchain, Bmodel=dl.bg_hom3d)

# Simulate the dipolar signal
Vsim = Vmodel(mod=moddepth, reftime=tau1, conc=spinconc, contour=contour, persistence=persistence, scale=1) 

# Plot the dipolar signal
plt.figure(figsize=[10,3])
plt.subplot(121)
plt.plot(t,Vsim,'k')
plt.xlabel('t [μs]')
plt.ylabel('V(t)')

plt.subplot(122)
P = dl.dd_wormchain(r,contour,persistence)
plt.plot(r,P,'k')
plt.xlabel('r [nm]')
plt.ylabel('P(r)')

plt.tight_layout()
plt.show()




To more accurately simulate an experimental signal, we can add some noise randomly sampled from a Gaussian distribution with a standard deviation of $\sigma$=0.005.

In [ ]:
# Add white Gaussian noise to the simulated signal
Vnoisy = Vsim + dl.whitegaussnoise(t,std=0.005,seed=1)

# Plot the dipolar signal
plt.plot(t,Vnoisy,'k.')
plt.xlabel('t [μs]')
plt.ylabel('V(t)')
plt.show()


--------------------

## Hands-on #2: Simulating a 4-pulse DEER signal

![title](https://jeschkelab.github.io/DeerLab/_images/sequence_4pdeer.svg)

We want to study via simulations, what kind of 4-pulse DEER signals we can expect from a ``75μM`` solution of linear biradical molecules. The distribution of interspin distances in these molecules is well approximated by a Gaussian model with a mean distance of ``4nm`` and a standard deviation of ``0.2nm``. We will set our 4-pulse DEER experiments with $\tau_1$=0.3μs and $\tau_2$=3μs, record our signal using ``16ns`` steps, and expect our modulation depth to be roughly ``15%``. 
 
Note:
- The 4-pulse DEER main contribution its refocusing time at $t=\tau_1$.
- The spin concentration is (ideally) twice the molecular concentration

In [ ]:

# Construct the dipolar time evolution vector
tau1 = 0.3 # μs
tau2 = 3   # μs
dt = 0.016 # μs
t = np.arange(0,tau1+tau2,dt)

# Construct the vector of interspin distances
rmin = 1  # nm
rmax = 10 # nm
dr = 0.05 # nm
r = np.arange(rmin,rmax,dr)

# <your script goes here>

----------------

## 3. Dipolar models with multiple pathway contributions

Dipolar signals can be described as a combination of contributions arising from different dipolar pathways. The intramolecular contribution consists of a sum and the intermolecular contribution consists of a product over individual dipolar pathway contributions. Each pathway has a characteristic refocusing time (point in time ``t`` where the pathway's contribution is largest) and amplitude. 

For example, for the 4-pulse DEER experiment, the possible dipolar pathways are:

<img src="https://github.com/JeschkeLab/DeerLabWorkshop2022/raw/main/images/pathways_scheme.png" alt="drawing" style="width:500px;"/>


In this demonstration we will see how to simulate a 4-pulse DEER signal with additional pathway contributions due to, e.g., pulse excitation overlap. These additional contributions have their largest contributions at the edges of the signal. Specifically the refocusing times of these pathways are: 

| Pathway | Refocusing time | Description | 
| :---: | ------ | -------------- |
| 1 | $t = \tau_1$ | Main contribution |
| 2 | $t = \tau_1+\tau_2$ | "2+1" contribution | 
| 3 | $t = 0$  | "Inverse 2+1" contribution | 
| 4 | $t = \tau_2$ | No description  | 

For the simulation we will use $\tau_1$ = 0.5μs and $\tau_2$=6μs using 8ns steps, with a Gaussian distance distribution.

In [ ]:
rmin = 1  # nm
rmax = 10 # nm
dr = 0.05 # nm
r = np.arange(rmin,rmax,dr)

tau1 = 0.5 # μs
tau2 = 6.0 # μs
dt = 0.008 # μs
t = np.arange(0,tau1+tau2,dt)

# Construct a three-pathway pathway model
Vmodel = dl.dipolarmodel(t,r,Pmodel=dl.dd_gauss,npathways=4)

print(Vmodel)


We will assume that the main contribution has a modulation depth (amplitude) of 25% while the contributions from pathways 2 and 3 have one 1/8th of its amplitude, and pathway 4 to have a negligible amplitude. We will use a spin concentration of 35μM and an average interspin distance of 4nm with a standard deviation of 0.3nm.  

In [ ]:

# Pathways refocusing times
reftime1 = tau1
reftime2 = tau1+tau2 
reftime3 = 0
reftime4 = tau2

# Pathway amplitudes
lam1 = 0.25
lam2 = lam1/8
lam3 = lam1/8
lam4 = 0 

# Other parameters
conc = 35 # μM
rmean = 4 # nm 
rstd = 0.3 # nm
scale = 1

Vsim = Vmodel(mean=rmean, std=rstd, conc=conc, scale=scale, 
                lam1=lam1,lam2=lam2, lam3=lam3, lam4=lam4,
                reftime1=reftime1,reftime2=reftime2,reftime3=reftime3,reftime4=reftime4)

# Add some noise
Vsim = Vsim + dl.whitegaussnoise(t,0.004,seed=1)

# Plot the dipolar signal
plt.plot(t,Vsim,'k.')
plt.xlabel('t [μs]')
plt.ylabel('V(t)')
plt.show()

-----------------